In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import time
import os
import copy
import torch.nn.functional as F


mean, std = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

preprocess_augment = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

dataset = datasets.ImageFolder(root = "../../share_dataset/six_objects/stimuli_objects/", transform=preprocess_augment)

In [2]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=True , num_workers=2)

In [3]:
# for i in data_loader:
#     print(i)
#     break

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [5]:
model = torchvision.models.alexnet(pretrained = False)

model.classifier[6] = nn.Linear(in_features=4096,out_features=6,bias=True)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
params_to_update = model.parameters()
optimizer = optim.SGD(params_to_update , lr=0.001, momentum=0.9)

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, weights_name='weight_save', is_inception=False):
    since = time.time()

    val_acc_history = []
    loss_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start = time.time()

        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        # loss2 = criterion(aux4a, labels)
                        # loss3 = criterion(aux4d, labels)
                        loss = loss1 #+ (0.3 * loss2) + (0.3 * loss3)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    # Backpropagate only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # Gather our summary statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_end = time.time()
            
            elapsed_epoch = epoch_end - epoch_start

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print("Epoch time taken: ", elapsed_epoch)

            # If this is the best model on the validation set so far, deep copy it
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), weights_name + ".pth")
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if phase == 'train':
                loss_acc_history.append(epoch_loss)

        print()

    # Output summary statistics, load the best weight set, and return results
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, loss_acc_history

In [9]:
dataloaders = { 'train': data_loader}
best_model, val_acc_history, loss_acc_history = train_model(model, dataloaders, criterion, optimizer, 9000, 'google_softmax_lr_0.001_bestsofar', is_inception=True)

Epoch 0/8999
----------
train Loss: 1.7924 Acc: 0.1583
Epoch time taken:  0.5950281620025635

Epoch 1/8999
----------
train Loss: 1.7921 Acc: 0.1667
Epoch time taken:  0.6442277431488037

Epoch 2/8999
----------
train Loss: 1.7919 Acc: 0.1667
Epoch time taken:  0.596604585647583

Epoch 3/8999
----------
train Loss: 1.7917 Acc: 0.1667
Epoch time taken:  0.6810956001281738

Epoch 4/8999
----------
train Loss: 1.7920 Acc: 0.1667
Epoch time taken:  0.7555420398712158

Epoch 5/8999
----------
train Loss: 1.7919 Acc: 0.1667
Epoch time taken:  0.6826198101043701

Epoch 6/8999
----------
train Loss: 1.7915 Acc: 0.1667
Epoch time taken:  0.6393406391143799

Epoch 7/8999
----------
train Loss: 1.7918 Acc: 0.1667
Epoch time taken:  0.665916919708252

Epoch 8/8999
----------
train Loss: 1.7917 Acc: 0.1667
Epoch time taken:  0.6456997394561768

Epoch 9/8999
----------
train Loss: 1.7916 Acc: 0.1667
Epoch time taken:  0.648155927658081

Epoch 10/8999
----------
train Loss: 1.7916 Acc: 0.1667
Epoch t